## Step output

### Output files

Output files of a step can be specified by step `output`. Whereas step `input` override or
change SoS provided variable `input` to produce one or more variable `_input`,
the step `output` specify `output` which should be `[]` if no output is generated
(or of interest). Similar to `input`, step output accepts strings, variables, expressions, and allows wildcard characters. For example, the following are acceptable output files

```python
output:  []

output:  'accepted_hits.bam'

output:  aligned_reads, bam_stats

output:  'aligned/*.bam'

output:  expand_pattern('aligned_{samples}.bam')
```

In the last example, function `expand_pattern` is used to contruct list of files from items of a sequence `samples`. In case of `input` loop, step `output` actually determines variable `_output` for each input loop. For example, the following step accepts one or more bam files and index them using command `samtools index`. The input files are passed one
by one so generate multiple `output` files (`'${_input}.bai'`). 

```python
[10]
input:
	bamfiles, group_by='single'

output:
	'${_input}.bai'

run('''samtools index ${_input} ''')
```

The use of variable `output` in this scenario is discouraged because `output`, as the collection of all `_output` increases with each input group. The accumulation only happens when `_output` and `output` are different so `output` will stay the same if all output files are specified as in the following example

```python
output: [x + '.bai' for x in bamfiles]
```

### `dynamic` output files

Similar to the cases with [dynamic input files](#dynamically-determined-input-files-function-dynamic), the
output of some steps could also not be determined beforehand. For example, with the following script that generates `html` files that cannot be determined during dry run, 

```python
[0]
run:
    rm -f *.html

[10]
input: []
output: '*.html'

import random
for i in range(5):
    run('touch result_${random.randint(1, 20)}.html')
```

SoS will determine that you do not have any output file (no `*.html` file) and produce the following output

```bash
$ sos run test.sos
INFO: Execute default_0:
INFO: input:   []
INFO: output:  unspecified
INFO: Execute default_10:
INFO: input:   []
WARNING: *.html does not expand to any valid file.
INFO: output:  []
```

In this case, you will need to define the output as `dynamic` using

```
[0]
run:
    rm -f *.html

[10]
input: []
output: dynamic('*.html')

import random
for i in range(5):
    run('touch result_${random.randint(1, 20)}.html')
```

so that SoS knows that the output can only be determined after the completion of the step. Because of this, variable `output` is unavailable to the step process.